In [2]:
import os
import cv2
import random
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

#Paso1
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

#Paso2
dataset_folder = "dataset"
all_images = []
for pose_folder in os.listdir(dataset_folder):
    pose_path = os.path.join(dataset_folder, pose_folder)
    if os.path.isdir(pose_path):
        images = [os.path.join(pose_path, img) for img in os.listdir(pose_path) if img.endswith(('.jpg', '.png'))]
        all_images.extend(images)
#Paso3
selected_images = random.sample(all_images, 10)

#Paso4
for img_path in selected_images:
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    #Paso5
    if results.pose_landmarks:
        annotated_image = image_rgb.copy()
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=3, circle_radius=3),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )           
        plt.figure(figsize=(6, 6))
        plt.imshow(annotated_image)
        plt.title(f'Pose Estimation - {os.path.basename(img_path)}')
        plt.axis('off')
        plt.show()
#Paso6
pose.close()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\USUARIO\miniconda

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
'''
-Paso 1: Inicialización del Modelo MediaPipe Pose
Se crea un objeto Pose de MediaPipe, que detectará los puntos clave del cuerpo en cada imagen.
static_image_mode=True: Indica que se procesan imágenes individuales.
min_detection_confidence=0.5: Establece la confianza mínima para detectar la pose.
También se define mp_drawing para dibujar el esqueleto sobre la imagen.

-Paso 2: Obtener Todas las Imágenes del Dataset
El código recorre todas las carpetas dentro de dataset/, verifica que realmente sean carpetas y almacena las rutas de todas las imágenes en una lista (all_images).

-Paso 3: Seleccionar 10 Imágenes Aleatorias
Si hay más de 10 imágenes disponibles, se eligen 10 aleatoriamente con random.sample(). Si hay menos, se seleccionan todas.

-Paso 4: Procesar Cada Imagen Seleccionada
Para cada imagen:
    Se carga con cv2.imread().
    Se convierte a formato RGB (cv2.COLOR_BGR2RGB) para que MediaPipe pueda procesarla correctamente.
    Se usa pose.process(image_rgb), que devuelve una lista de puntos clave si se detecta una persona en la imagen.

-Paso 5: Dibujar el Esqueleto Sobre la Imagen
Si se detecta una pose:
    Se copian los datos de la imagen original.
    Se dibujan los puntos clave en verde.
    Se dibujan las conexiones entre puntos (brazos, piernas, torso) en rojo.

-Paso 6: Cerrar el Modelo de MediaPipe
Al finalizar, se libera el modelo con pose.close().
'''

'\n-Paso 1: Inicialización del Modelo MediaPipe Pose\nSe crea un objeto Pose de MediaPipe, que detectará los puntos clave del cuerpo en cada imagen.\nstatic_image_mode=True: Indica que se procesan imágenes individuales.\nmin_detection_confidence=0.5: Establece la confianza mínima para detectar la pose.\nTambién se define mp_drawing para dibujar el esqueleto sobre la imagen.\n\n-Paso 2: Obtener Todas las Imágenes del Dataset\nEl código recorre todas las carpetas dentro de dataset/, verifica que realmente sean carpetas y almacena las rutas de todas las imágenes en una lista (all_images).\n\n-Paso 3: Seleccionar 10 Imágenes Aleatorias\nSi hay más de 10 imágenes disponibles, se eligen 10 aleatoriamente con random.sample(). Si hay menos, se seleccionan todas.\n\n-Paso 4: Procesar Cada Imagen Seleccionada\nPara cada imagen:\n    Se carga con cv2.imread().\n    Se convierte a formato RGB (cv2.COLOR_BGR2RGB) para que MediaPipe pueda procesarla correctamente.\n    Se usa pose.process(image_rgb)

In [2]:
!pip install pandas

  Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [2]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

dataset_folder = "dataset"
data = []
labels = []

for pose_folder in os.listdir(dataset_folder):
    pose_path = os.path.join(dataset_folder, pose_folder)
    if os.path.isdir(pose_path):
        for img_name in os.listdir(pose_path):
            if img_name.endswith(('.jpg', '.png')):
                img_path = os.path.join(pose_path, img_name)
                image = cv2.imread(img_path)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image_rgb)

                if results.pose_landmarks:
                    landmarks = []
                    for lm in results.pose_landmarks.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])

                    data.append(landmarks)
                    labels.append(pose_folder)

df = pd.DataFrame(data)
df['label'] = labels
df.to_csv("landmarks_dataset.csv", index=False)

print("Landmarks guardados en landmarks_dataset.csv")

pose.close()


Landmarks guardados en landmarks_dataset.csv


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Cargar dataset
df = pd.read_csv("landmarks_dataset.csv")

# Separar características (X) y etiquetas (y)
X = df.iloc[:, :-1].values  # Todas las columnas menos la última
y = df["label"].values  # Última columna

# Convertir etiquetas a números
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # Convierte los nombres de poses en números

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\USUARIO\miniconda

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USUARIO\miniconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USUARIO\miniconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\USUARIO\miniconda

AttributeError: _ARRAY_API not found